<a href="https://colab.research.google.com/github/satvik314/ai_experiments/blob/main/function_calling_lc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.prompts import ChatPromptTemplate

In [ ]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass()

··········


In [ ]:
llm = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo-0613")

In [ ]:
schema = {
    "properties" : {
        "sentiment" : {"type" : "string"},
        "stars" : {"type" : "integer"},
        "language" : {"type" : "string"}
    }
}

In [ ]:
chain = create_tagging_chain(schema, llm)

In [ ]:
chain.prompt.messages[0].prompt

PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='Extract the desired information from the following passage.\n\nPassage:\n{input}\n', template_format='f-string', validate_template=True)

In [ ]:
print(chain.prompt.messages[0].prompt.template)

Extract the desired information from the following passage.

Passage:
{input}



In [ ]:
chain.llm_kwargs

{'functions': [{'name': 'information_extraction',
   'description': 'Extracts the relevant information from the passage.',
   'parameters': {'type': 'object',
    'properties': {'sentiment': {'title': 'sentiment', 'type': 'string'},
     'stars': {'title': 'stars', 'type': 'integer'},
     'language': {'title': 'language', 'type': 'string'}},
    'required': []}}],
 'function_call': {'name': 'information_extraction'}}

In [ ]:
chain.output_parser

JsonOutputFunctionsParser()

In [ ]:
review_01 = "Starts off kind of slow, but builds up quickly to give the reader a good understanding of how things unfolded. I’m anticipating a second book, and can’t wait for the rest of the story to be unveiled!"
chain.run(review_01)

{'sentiment': 'positive', 'stars': 4}

In [ ]:
review_02 = """It's the BIGGEST CON \n
This book has NOT been security checked for correct information , I feel Meghan and Harry are laughing at me for buying this book , And putting my money in there pockets. Harry can't come up with any dates, says he can't remember the dates of anything , Said he was At college on a Hot summers day when the phone call came about his Great Great Grandmother passing , He was not at school he was Skiing in Switzerland with his Brother ( I remember this) And the papers have come back with the proof. Harry says that his mother bought him an XBox for his 13th birthday in advance of her death that her sister brought to the school for him, again not true the XBox didn't come out till 4 years later . There is so many holes in the book . Every other page has something on it that didn't happen. He takes NO reasonability for ANYthing even when the evidence is staring him in his face. This book is a BIG con, don't waste your money."""

chain.run(review_02)

{'sentiment': 'negative'}

In [ ]:
schema = {
    "properties" : {
        "sentiment" : {"type" : "string", "enum" : ["positive", "neutral", "negative"]},
        "stars" : {
            "type" : "integer",
            "enum" : [1,2,3,4,5]
        },
        "language" : {
            "type" : "string",
            "enum" : ["spanish", "english", "french", "german", "italian"]
        }
    },
    "required" : ["language", "sentiment", "stars"]
}

In [ ]:
chain = create_tagging_chain(schema, llm)

In [ ]:
chain.run(review_01)

{'sentiment': 'positive', 'stars': 4, 'language': 'english'}

In [ ]:
chain.run(review_02)

{'sentiment': 'negative', 'stars': 1, 'language': 'english'}

## Using Pydantic

In [ ]:
from enum import Enum
from pydantic import BaseModel, Field

In [ ]:
class Tags(BaseModel):
  sentiment : str = Field(..., enum = ["positive", "neutral", "negative"])
  stars : int = Field(
      ...,
      description = "describes the numbers of stars given by a reviewer on Amazon",
      enum = [1,2,3,4,5]
  )
  language : str = Field(
      ...,
      enum = ['spanish', 'english', 'french', 'german', 'italian']
  )


In [ ]:
chain = create_tagging_chain_pydantic(Tags, llm)

In [ ]:
res = chain.run(review_01)

In [ ]:
print(res)

sentiment='positive' stars=4 language='english'


In [ ]:
res.sentiment

'positive'

## Extraction, NER

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate

In [ ]:
tc_01 = """Reddit CEO Steve Huffman is not backing down amid protests against API changes made by the platform. In interviews with The Verge, NBC News and NPR, Huffman defended business decisions made by the company to charge third-party apps saying the API wasn’t designed to support these clients.

The Reddit co-founder also talked about protesting moderators, changing site rules, and profitability in these interviews. The platform is facing one of the strongest backlashes from the community, but the CEO seemingly doesn’t want to budge.

What’s happening at Reddit?
In April, Reddit announced that it is going to charge for its API, but didn’t announce any pricing. Earlier this month, Christian Selig, the developer of a popular Reddit client for iOS called Apollo, posted that he had a call with Reddit. API pricing quoted by them could cost him $20 million a year to run the app. Selig later said that, because the social network is not ready to make any changes to the pricing structure, he is forced to shut down Apollo. Other third-party developers of clients like Reddit is Fun and Relay for Reddit also said that they will shut down their apps on June 30.

The only exception Reddit made was to allow free access to the API to non-commercial apps providing accessibility features. The company has made deals with apps like RedReader, Dystopia, and Luna and given them exemptions from its “large-scale pricing terms.”

Thousands of subreddits went dark starting June 12 to protest those changes — it caused a brief outage as well. Meanwhile, Huffman took a strong stance in his AMA and took a dig at Apollo and Selig. As moderators didn’t see anything changing, many subreddits decided to extend the blackout."""

In [ ]:
llm = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo-0613")

In [ ]:
schema = {
    "properties" : {
        "person_name" : {"type" : "string"},
        "startup" : {"type" : "string"},
        "news_outlet" : {"type" : "string"},
        "app_name" : {"type" : "string"},
        "month" : {"type" : "string"}
    },
    "required" : ["person_name", "startup"]
}

In [ ]:
chain = create_extraction_chain(schema, llm)

In [ ]:
chain.prompt.messages[0]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='Extract and save the relevant entities mentioned in the following passage together with their properties.\n\nPassage:\n{input}\n', template_format='f-string', validate_template=True), additional_kwargs={})

In [ ]:
print(chain.prompt.messages[0].prompt.template)

Extract and save the relevant entities mentioned in the following passage together with their properties.

Passage:
{input}



In [ ]:
chain.llm_kwargs

{'functions': [{'name': 'information_extraction',
   'description': 'Extracts the relevant information from the passage.',
   'parameters': {'type': 'object',
    'properties': {'info': {'type': 'array',
      'items': {'type': 'object',
       'properties': {'person_name': {'title': 'person_name',
         'type': 'string'},
        'startup': {'title': 'startup', 'type': 'string'},
        'news_outlet': {'title': 'news_outlet', 'type': 'string'},
        'app_name': {'title': 'app_name', 'type': 'string'},
        'month': {'title': 'month', 'type': 'string'}},
       'required': ['person_name', 'startup']}}},
    'required': ['info']}}],
 'function_call': {'name': 'information_extraction'}}

In [ ]:
chain.output_parser

JsonKeyOutputFunctionsParser(key_name='info')

In [ ]:
response = chain.run(tc_01)
response

[{'person_name': 'Steve Huffman',
  'startup': 'Reddit',
  'news_outlet': 'The Verge',
  'app_name': 'Apollo',
  'month': 'April'},
 {'person_name': 'Christian Selig',
  'startup': 'Apollo',
  'news_outlet': 'N/A',
  'app_name': 'Apollo',
  'month': 'April'},
 {'person_name': 'Christian Selig',
  'startup': 'Apollo',
  'news_outlet': 'N/A',
  'app_name': 'Apollo',
  'month': 'June'},
 {'person_name': 'Reddit',
  'startup': 'Reddit',
  'news_outlet': 'N/A',
  'app_name': 'Reddit is Fun',
  'month': 'June'},
 {'person_name': 'Reddit',
  'startup': 'Reddit',
  'news_outlet': 'N/A',
  'app_name': 'Relay for Reddit',
  'month': 'June'}]

In [ ]:
def sort_objects(obj_list):
    # Initialize empty lists for each category
    people = []
    startups = []
    news_outlets = []
    apps = []
    months = []

    # Loop through each dictionary in the list
    for obj in obj_list:
        # Add each value to the appropriate list, if the key exists in the dictionary and the value is not already in the list
        if 'person_name' in obj and obj['person_name'] not in people:
            people.append(obj['person_name'])
        if 'startup' in obj and obj['startup'] not in startups:
            startups.append(obj['startup'])
        if 'news_outlet' in obj and obj['news_outlet'] not in news_outlets:
            news_outlets.append(obj['news_outlet'])
        if 'app_name' in obj and obj['app_name'] not in apps:
            apps.append(obj['app_name'])
        if 'month' in obj and obj['month'] not in months:
            months.append(obj['month'])

    # Return the lists as a dictionary
    return {'people': people, 'startups': startups, 'news_outlets': news_outlets, 'apps': apps, 'months': months}

In [ ]:
sort_objects(response)

{'people': ['Steve Huffman', 'Christian Selig', 'Reddit'],
 'startups': ['Reddit', 'Apollo'],
 'news_outlets': ['The Verge', 'N/A'],
 'apps': ['Apollo', 'Reddit is Fun', 'Relay for Reddit'],
 'months': ['April', 'June']}

In [ ]:
from pydantic import BaseModel

class NER(BaseModel):
  person_name : str
  startup : str
  news_outlet : str
  app_name : str
  month : str

In [ ]:
chain = create_extraction_chain_pydantic(NER, llm)

In [ ]:
res = chain.run(tc_01)

In [ ]:
print(res)

[NER(person_name='Steve Huffman', startup='Reddit', news_outlet='The Verge', app_name='Apollo', month='April'), NER(person_name='Christian Selig', startup='Apollo', news_outlet='N/A', app_name='Apollo', month='April'), NER(person_name='Christian Selig', startup='Apollo', news_outlet='N/A', app_name='Apollo', month='June'), NER(person_name='Reddit', startup='Reddit', news_outlet='N/A', app_name='Reddit is Fun', month='June'), NER(person_name='Reddit', startup='Reddit', news_outlet='N/A', app_name='Relay for Reddit', month='June')]


In [ ]:
res[0].person_name

'Steve Huffman'